In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# Load MNIST data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
# Define network parameters
n_input = 784  # 28x28 pixels
n_hidden1 = 128  # First hidden layer neurons
n_hidden2 = 64   # Second hidden layer neurons
n_output = 10    # 10 classes (digits 0-9)
learning_rate = 0.01
n_epochs = 10
batch_size = 100

In [ ]:
# Define placeholders
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_output])

# Initialize weights and biases
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
    'h2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
    'out': tf.Variable(tf.random_normal([n_hidden2, n_output]))
}

biases = {
    'b1': tf.Variable(tf.zeros([n_hidden1])),
    'b2': tf.Variable(tf.zeros([n_hidden2])),
    'out': tf.Variable(tf.zeros([n_output]))
}

# Define the neural network model
def neural_network(x):
    layer1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer1 = tf.nn.relu(layer1)
    
    layer2 = tf.add(tf.matmul(layer1, weights['h2']), biases['b2'])
    layer2 = tf.nn.relu(layer2)
    
    output_layer = tf.add(tf.matmul(layer2, weights['out']), biases['out'])
    return output_layer

# Compute predictions and loss
logits = neural_network(X)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))

# Define optimizer for backpropagation
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# Evaluate accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize variables
init = tf.global_variables_initializer()

In [ ]:
# Train the model
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        total_batch = int(mnist.train.num_examples / batch_size)
        avg_loss = 0
        
        for _ in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, batch_loss = sess.run([optimizer, loss], feed_dict={X: batch_x, Y: batch_y})
            avg_loss += batch_loss / total_batch
        
        train_acc = sess.run(accuracy, feed_dict={X: mnist.train.images, Y: mnist.train.labels})
        print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}, Training Accuracy: {train_acc:.4f}")
    
    # Test accuracy
    test_acc = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels})
    print(f"Test Accuracy: {test_acc:.4f}")